In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.2 MB/s eta 0:00:00


In [2]:
import sqlite3
import random
from faker import Faker # Using Faker ensures the data is synthetic, original, and highly realistic
from datetime import datetime, timedelta

# --- Configuration ---
DATABASE_NAME = 'Financial_Transaction_DB.sqlite'
NUM_BRANCHES = 10
NUM_CUSTOMERS = 2000
NUM_ACCOUNTS = 3000  # More accounts than customers for realism (some customers have multiple)
NUM_TRANSACTIONS = 50000

# Initialize Faker
fake = Faker('en_GB')

# --- Data Generation Functions ---

def generate_branches(num):
    """Generates branch records."""
    branches = []
    tiers = ['Tier 1 - HQ', 'Tier 2 - Regional', 'Tier 3 - Local']

    for i in range(1, num + 1):
        branch_id = i
        location = fake.city() + ' Branch'
        # Branch_Tier (Ordinal Data)
        branch_tier = random.choice(tiers)
        # Assets (Ratio Data)
        assets = round(random.uniform(5_000_000, 500_000_000), 2)

        branches.append((branch_id, location, branch_tier, assets))
    return branches

def generate_customers(num, num_branches):
    """Generates customer records."""
    customers = []

    for i in range(1, num + 1):
        customer_id = i
        name = fake.name()
        email = fake.unique.email()
        # Account_Type (Nominal Data)
        account_type = random.choice(['Personal', 'Business', 'Joint'])
        # Risk_Category (Ordinal Data)
        # Low < Medium < High
        risk_category = random.choice(['Low', 'Medium', 'High'])
        # Ratio: DateJoined (Ratio from system start)
        date_joined = fake.date_between(start_date='-10y', end_date='today').isoformat()
        # Foreign Key to Branches
        branch_id = random.randint(1, num_branches)

        customers.append((customer_id, branch_id, name, email, account_type, risk_category, date_joined))
    return customers

def generate_accounts(num, num_customers):
    """Generates account records, linked to customers, ensuring unique (Customer, Currency) pairs."""
    accounts = []
    currencies = ['GBP', 'EUR', 'USD']
    used_pairs = set() # Set to track used (CustomerID, Currency) pairs

    i = 1
    # Loop until we generate the required number of *unique* accounts
    while len(accounts) < num:
        account_id = 100000 + i

        # --- Randomly select a customer and currency ---
        customer_id = random.randint(1, num_customers)
        currency = random.choice(currencies)

        pair = (customer_id, currency)

        # Rationale: Check the Composite Key rule before insertion.
        # This prevents the "UNIQUE constraint failed" error.
        if pair not in used_pairs:
            # Generate the rest of the data only if the pair is unique
            interest_rate = round(random.uniform(0.1, 5.0), 2)
            balance = round(random.uniform(50.00, 500_000.00), 2)

            accounts.append((account_id, customer_id, currency, interest_rate, balance))
            used_pairs.add(pair)
            i += 1 # Only increment counter if a unique account was created

    return accounts

def generate_transactions(num_accounts, num_transactions):
    """Generates transaction records."""
    transactions = []
    transaction_types = ['Deposit', 'Withdrawal', 'Transfer', 'Payment']
        # Ensures chronological transaction ordering and prevents date outliers, supporting the Interval data type requirement
    start_date = datetime.now() - timedelta(days=365)

    for i in range(1, num_transactions + 1):
        transaction_id = i
        # Foreign Key
        account_id = random.randint(100001, 100000 + num_accounts)

        # Transaction_Date (Interval Data - Arbitrary zero in time)
        transaction_date = start_date + timedelta(days=random.randint(1, 365), seconds=random.randint(0, 86400))
        transaction_date_str = transaction_date.isoformat().replace('T', ' ')

        transaction_type = random.choice(transaction_types)
        # Amount (Ratio Data - Meaningful zero)
        amount = round(random.uniform(0.01, 10000.00), 2)

        # Introducing deliberate missing/duplicate data
        description = fake.word().capitalize()
        if random.random() < 0.03: # 3% chance of missing data (e.g., failed logging)
             description = None

        transactions.append((transaction_id, account_id, transaction_date_str, transaction_type, amount, description))

    # --- Introduce Deliberate Duplicate Transaction Data ---
    for _ in range(int(NUM_TRANSACTIONS * 0.01)):
        duplicate_txn = random.choice(transactions)
        # Tweak the TransactionID and Date to avoid PK violation but keep data duplicated
        new_id = transactions[-1][0] + 1
        # Slight time change for date realism
        new_date = datetime.strptime(duplicate_txn[2], '%Y-%m-%d %H:%M:%S.%f') + timedelta(seconds=1)

        # Create a new record that is essentially a duplicate
        transactions.append((new_id, duplicate_txn[1], new_date.isoformat().replace('T', ' '), *duplicate_txn[3:]))

    return transactions

# --- Database Creation and Schema Definition ---

def create_database(db_name):
    """Connects to and creates the tables in the SQLite database."""
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # 1. Branches Table
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Branches (
        BranchID INTEGER PRIMARY KEY,
        Location TEXT NOT NULL UNIQUE,
        Branch_Tier TEXT CHECK(Branch_Tier IN ('Tier 1 - HQ', 'Tier 2 - Regional', 'Tier 3 - Local')),
        Assets REAL NOT NULL CHECK(Assets >= 0)
    );
    """)

    # 2. Customers Table
    # Account_Type (Nominal), Risk_Category (Ordinal)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerID INTEGER PRIMARY KEY,
        BranchID INTEGER NOT NULL,
        Name TEXT NOT NULL,
        Email TEXT UNIQUE NOT NULL,
        Account_Type TEXT NOT NULL CHECK(Account_Type IN ('Personal', 'Business', 'Joint')),
        Risk_Category TEXT CHECK(Risk_Category IN ('Low', 'Medium', 'High')),
        DateJoined TEXT NOT NULL,

        FOREIGN KEY (BranchID) REFERENCES Branches(BranchID)
    );
    """)

    # 3. Accounts Table (Composite and Foreign Keys)
    # Currency (Nominal), Interest_Rate (Ratio), Balance (Ratio)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Accounts (
        AccountID INTEGER PRIMARY KEY,
        CustomerID INTEGER NOT NULL,
        Currency TEXT NOT NULL CHECK(Currency IN ('GBP', 'EUR', 'USD')),
        Interest_Rate REAL NOT NULL CHECK(Interest_Rate >= 0),
        Balance REAL NOT NULL,

        -- Using a COMPOSITE KEY to enforce a customer can only have one account per currency
        -- Note: SQLite does not strictly enforce composite key on its own without UNIQUE,
        -- but this represents the logical key structure.
        UNIQUE (CustomerID, Currency)

        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    );
    """)

    # 4. Transactions Table (Over 1000 rows)
    # Transaction_Date (Interval), Amount (Ratio)
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS Transactions (
        TransactionID INTEGER PRIMARY KEY,
        AccountID INTEGER NOT NULL,
        Transaction_Date TEXT NOT NULL,
        Transaction_Type TEXT NOT NULL CHECK(Transaction_Type IN ('Deposit', 'Withdrawal', 'Transfer', 'Payment')),
        Amount REAL NOT NULL CHECK(Amount > 0.0),
        Description TEXT, -- Allows for missing data

        FOREIGN KEY (AccountID) REFERENCES Accounts(AccountID)
    );
    """)

    conn.commit()
    return conn

# --- Data Insertion ---

def insert_data(conn):
    """Generates and inserts data into the created tables."""
    cursor = conn.cursor()

    # 1. Branches
    print(f"Generating {NUM_BRANCHES} branches...")
    branch_data = generate_branches(NUM_BRANCHES)
    cursor.executemany("INSERT INTO Branches VALUES (?, ?, ?, ?)", branch_data)

    # 2. Customers
    print(f"Generating {NUM_CUSTOMERS} customers...")
    customer_data = generate_customers(NUM_CUSTOMERS, NUM_BRANCHES)
    cursor.executemany("INSERT INTO Customers VALUES (?, ?, ?, ?, ?, ?, ?)", customer_data)

    # 3. Accounts
    print(f"Generating {NUM_ACCOUNTS} accounts...")
    account_data = generate_accounts(NUM_ACCOUNTS, NUM_CUSTOMERS)
    cursor.executemany("INSERT INTO Accounts VALUES (?, ?, ?, ?, ?)", account_data)

    # 4. Transactions (The large table)
    print(f"Generating {NUM_TRANSACTIONS} transactions...")
    transaction_data = generate_transactions(NUM_ACCOUNTS, NUM_TRANSACTIONS)
    print(f"Total transactions to insert (including duplicates/missing data): {len(transaction_data)}")
    cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?, ?)", transaction_data)

    conn.commit()
    print("Data insertion complete.")


# --- Main Execution ---

if __name__ == "__main__":
    try:
        print(f"Starting database creation: {DATABASE_NAME}")
        conn = create_database(DATABASE_NAME)
        insert_data(conn)
        conn.close()
        print("Database fully created and populated.")
    except Exception as e:
        print(f"An error occurred: {e}")

Starting database creation: Financial_Transaction_DB.sqlite
Generating 10 branches...
Generating 2000 customers...
Generating 3000 accounts...
Generating 50000 transactions...
Total transactions to insert (including duplicates/missing data): 50500
Data insertion complete.
Database fully created and populated.
